In [29]:
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.layers import Dense, Conv1D, MaxPool2D, Flatten, Embedding
from keras.layers import Dropout, Flatten, GlobalMaxPool1D, Activation
from keras.models import Sequential

In [30]:
max_features = 10000
maxlen = 500
batch_size = 32

imdb데이터셋을 사용하기전 전처리가 필요하다.
Keras의 전처리 함수인 pad_sequences를 사용 
imdb에서 불러올 크기는 10000 으로 지정했다

In [31]:
(x_train, y_train), (x_label, y_label) = imdb.load_data(num_words = max_features)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_label = sequence.pad_sequences(x_label, maxlen=maxlen)

결과가 이진으로 나뉘어지는 문제다. 따라서 Dense이후에 sigmoid를 붙여 최종적인 결과를 낸다.
 
레이어를 선형으로 쌓기위해 Sequential 모델을 만든다.
Embedding에서의 첫번째 파라미터는 input_dim이다. 이는 imdb load시의 크기와 같아야하므로
max_features로 둔다. 두번째 인자는 output_dim으로, 50차원 공간에 두는 것이다.
input_length로 maxlen인 500을 두었으며 문장길이를 지정하였다.
Dropout은 과적합 방지를 위해 사용하였다. 입력데이터 * 0.2만큼 입력값으로 사용하지 않는다.
GlobalMaxPool1d 를 사용하였는데 이는 3차원을 2차원으로 변형해준다.

Dense로 출력뉴런의 수를 1로 정한다. 입력뉴런은 GlobalMaxPool1d에서 나온 2이다
은닉층에 relu레이어를 둔다..
이 문제는 **positive negative를 판별하는 문제기에** 마지막으로 sigmoid함수를 이용한다.

In [24]:
model = Sequential()
model.add(Embedding(max_features, 50, input_length=maxlen))
# 50dimension
model.add(Dropout(0.2))
model.add(Conv1D(250, 3,
                padding='valid',
                activation='relu',
                strides=1))
model.add(GlobalMaxPool1D())
model.add(Dense(250))
model.add(Dropout(0.2))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

퍼포먼스를 위해 옵티마이저로 rmsprop를 사용하였다.

In [25]:
model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['acc'])

W0803 11:47:18.364218 140120434202432 deprecation_wrapper.py:119] From /home/aaronroh/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0803 11:47:18.394880 140120434202432 deprecation_wrapper.py:119] From /home/aaronroh/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0803 11:47:18.399853 140120434202432 deprecation.py:323] From /home/aaronroh/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


모델을 가지고 훈련을 하는데 epoch는 25, 훈련은 0.8, 검증데이터셋은 0.2로 나누어 진행한다

In [26]:
history = model.fit(x_train, y_train,
                   epochs=25,
                   batch_size = 128,
                   validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/25
20000/20000 [==============================] - 31s 2ms/step - loss: 0.5295 - acc: 0.7166 - val_loss: 0.3544 - val_acc: 0.8464
Epoch 2/25
20000/20000 [==============================] - 30s 1ms/step - loss: 0.3090 - acc: 0.8683 - val_loss: 0.2813 - val_acc: 0.8822
Epoch 3/25
20000/20000 [==============================] - 33s 2ms/step - loss: 0.2353 - acc: 0.9053 - val_loss: 0.2599 - val_acc: 0.8956
Epoch 4/25
20000/20000 [==============================] - 36s 2ms/step - loss: 0.1835 - acc: 0.9303 - val_loss: 0.2821 - val_acc: 0.8888
Epoch 5/25
20000/20000 [==============================] - 36s 2ms/step - loss: 0.1512 - acc: 0.9439 - val_loss: 0.2637 - val_acc: 0.8986
Epoch 6/25
20000/20000 [==============================] - 33s 2ms/step - loss: 0.1197 - acc: 0.9561 - val_loss: 0.3584 - val_acc: 0.8720
Epoch 7/25
20000/20000 [==============================] - 31s 2ms/step - loss: 0.0944 - acc: 0.9652 - val_loss: 0.3113 - val_acc:

CNN 모델에 대한 요약이다.
레이어는 embedding-dropout-conv1d-global_max_pooling1d - dense - dropout - activation - dense - activation 로 구성되어있다

In [27]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 500, 50)           500000    
_________________________________________________________________
dropout_6 (Dropout)          (None, 500, 50)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 498, 250)          37750     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 250)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 250)               62750     
_________________________________________________________________
dropout_7 (Dropout)          (None, 250)               0         
_________________________________________________________________
activation_2 (Activation)    (None, 250)               0         
__________

정확도는 0.88296이다.

In [28]:
results = model.evaluate(x_label, y_label)
results

25000/25000 [==============================] - 8s 333us/step


[0.8321108200644143, 0.88296]